# Correct Crosswalk
ok - rewind a lot. within merge_data, there's this comment. After having the portfolio_organization_names merged in, we looked at what made the grain correct, which is name-portfolio_organization_name-service_date. That means that if you back up the lines before that...reading in columns like organization_name to begin with was what created this 1:m issue, which we now know how to handle.
 
```
df = df.assign(
        caltrans_district = df.caltrans_district.map(
            portfolio_utils.CALTRANS_DISTRICT_DICT
        )
    ).pipe(
        portfolio_utils.standardize_portfolio_organization_names, 
        PORTFOLIO_ORGANIZATIONS_DICT
    )
    # to aggregate up to organization, 
    # group by name-service_date-portfolio_organization_name
    # because name indicates different feeds, so we want to sum those.
```
Back up to this section right before: 

```
crosswalk_cols = [
       "schedule_gtfs_dataset_key",
       "name",
       "schedule_source_record_id",
       "base64_url",
       #"organization_source_record_id", # remove this column which gave us 1:m issue
       #"organization_name", # remove this column which gave us the 1:m issue
       "caltrans_district"
   ]
   df = time_series_utils.concatenate_datasets_across_dates(
       SCHED_GCS,
       FILE,
       date_list,
       data_type = "df",
       columns = crosswalk_cols
   )
```
continue pipe for caltrans district, 
continue pipe for portfolio_organization_name, which uses (schedule_gtfs_dataset)name to map the dictionary
 
This change actually addresses the core issue after we looked at it and determined that (schedule_gtfs_dataset)_name-portfolio_organization_name-service_date is the correct grain. Make this change and check how your routes look.
 
I got to this point because I looked at why your rows had City of Moorpark, City of Thousand Oaks, and that doesn't seem correct. 
In the yaml, there is one entry: Thousand Oaks Flex: City of Thousand Oaks, but when I read in monthly_route_schedule_vp, I did find Thousand Oaks Flex in there at all. Flex is not the problem. 
However, the presence of City of Thousand Oaks tells me organization_name is still somewhere, and that shouldn't be used because we've already moved away from it, hence never using it in the aggregation after.
It's showing up in merge_data because the crosswalk wasn't adjusted to remove it. We don't need those columns, it gave us issues already, and now it needs to be cleaned up and removed.
 

In [16]:
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
import yaml
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS
from segment_speed_utils import gtfs_schedule_wrangling, time_series_utils
from shared_utils import gtfs_utils_v2, catalog_utils, rt_dates, rt_utils, publish_utils,  portfolio_utils
readable_dict = OmegaConf.load("readable2.yml")
credentials, project = google.auth.default()

import _report_route_dir_visuals
import merge_data

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# portfolio_name = "City and County of San Francisco"
portfolio_name = "Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"

In [11]:
date_list = rt_dates.y2025_dates

## `concatenate_crosswalk_organization`

In [4]:
FILE = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk
    
crosswalk_cols = [
        "schedule_gtfs_dataset_key",
        "name",
        "schedule_source_record_id",
        "base64_url",
        #"organization_source_record_id",
        #"organization_name",
        "caltrans_district"
    ]

In [12]:
df = time_series_utils.concatenate_datasets_across_dates(
        SCHED_GCS,
        FILE,
        date_list,
        data_type = "df",
        columns = crosswalk_cols
    )

In [13]:
df.head(2)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,caltrans_district,service_date
0,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,recuWhPXfxMatv6rL,aHR0cDovL2FwcC5tZWNhdHJhbi5jb20vdXJiL3dzL2ZlZWQvYzJsMFpUMXplWFowTzJOc2FXVnVkRDF6Wld4bU8yVjRjR2x5WlQwN2RIbHdaVDFuZEdaek8ydGxlVDAwTWpjd056UTBaVFk0TlRBek9UTXlNREl4TURkak56STBNRFJrTXpZeU5UTTRNekkwWXpJMA==,05 - San Luis Obispo,2025-01-15
1,f4c3ea214214ee0d96f7646b3e9d69dc,SLO Peak Transit Schedule,rec0EeeizKvsEDfRQ,aHR0cDovL2RhdGEucGVha3RyYW5zaXQuY29tL3N0YXRpY2d0ZnMvMS9ndGZzLnppcA==,05 - San Luis Obispo,2025-01-15


In [17]:

with open(
    "../_shared_utils/shared_utils/portfolio_organization_name.yml", "r"
) as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [18]:
df = df.assign(
        caltrans_district = df.caltrans_district.map(
            portfolio_utils.CALTRANS_DISTRICT_DICT
        )
    ).pipe(
        portfolio_utils.standardize_portfolio_organization_names, 
        PORTFOLIO_ORGANIZATIONS_DICT
    )

In [19]:
df.head(2)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,caltrans_district,service_date,portfolio_organization_name
0,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,recuWhPXfxMatv6rL,aHR0cDovL2FwcC5tZWNhdHJhbi5jb20vdXJiL3dzL2ZlZWQvYzJsMFpUMXplWFowTzJOc2FXVnVkRDF6Wld4bU8yVjRjR2x5WlQwN2RIbHdaVDFuZEdaek8ydGxlVDAwTWpjd056UTBaVFk0TlRBek9UTXlNREl4TURkak56STBNRFJrTXpZeU5UTTRNekkwWXpJMA==,05 - San Luis Obispo / Santa Barbara,2025-01-15,City of Solvang
1,f4c3ea214214ee0d96f7646b3e9d69dc,SLO Peak Transit Schedule,rec0EeeizKvsEDfRQ,aHR0cDovL2RhdGEucGVha3RyYW5zaXQuY29tL3N0YXRpY2d0ZnMvMS9ndGZzLnppcA==,05 - San Luis Obispo / Santa Barbara,2025-01-15,San Luis Obispo Regional Transit Authority


In [20]:
df.loc[df.portfolio_organization_name == portfolio_name]

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,caltrans_district,service_date,portfolio_organization_name
162,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,2025-01-15,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
162,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,2025-02-12,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
159,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,2025-03-12,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
162,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,2025-04-16,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
158,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,2025-05-14,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"


In [21]:
df_sched = merge_data.concatenate_schedule_by_route_direction(date_list)
    
df_avg_speeds = merge_data.concatenate_speeds_by_route_direction(
        date_list
    )
    
df_rt_sched = merge_data.concatenate_rt_vs_schedule_by_route_direction(
        date_list
    )

In [22]:
df_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,service_date,is_express,is_ferry,is_rail,is_coverage,is_local,is_downtown_local,is_rapid,typology,name,combined_name,recent_combined_name,recent_route_id,route_primary_direction
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,all_day,63.50,0.92,2,0.08,2025-01-15,0.00,0.00,0.00,1.00,0.00,0.00,0.00,coverage,TCRTA TripShot Schedule,C70 LOOP__70,C70 LOOP 70,0177a66b-9f33-407d-a72e-776429fb73d4,Eastbound
1,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,all_day,63.50,5.98,2,0.08,2025-02-12,0.00,0.00,0.00,1.00,0.00,0.00,0.00,coverage,TCRTA TripShot Schedule,C70 LOOP__70,C70 LOOP 70,0177a66b-9f33-407d-a72e-776429fb73d4,Eastbound


In [23]:
df_avg_speeds.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,all_day,33.49,2025-04-16
1,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,peak,33.49,2025-04-16


In [24]:
df_rt_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,service_date
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,all_day,159,158,156.95,127.00,473,390,0,1,1,2,3.01,0.82,1.00,1.00,1.00,1.00,1.24,78.47,2025-01-15
1,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,NaN,all_day,161,159,158.92,127.00,479,378,0,1,1,2,3.01,0.79,1.00,1.00,1.00,1.00,1.25,79.46,2025-02-12


In [26]:
df2 = merge_data.merge_data_sources_by_route_direction(
        df_sched,
        df_rt_sched,
        df_avg_speeds,
        df
    )

## Check Ventura County Route 80 

In [27]:
df2.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'direction_id', 'time_period',
       'avg_scheduled_service_minutes', 'avg_stop_miles', 'n_scheduled_trips',
       'frequency', 'service_date', 'is_express', 'is_ferry', 'is_rail',
       'is_coverage', 'is_local', 'is_downtown_local', 'is_rapid', 'typology',
       'name', 'combined_name', 'recent_combined_name', 'recent_route_id',
       'route_primary_direction', 'minutes_atleast1_vp', 'minutes_atleast2_vp',
       'total_rt_service_minutes', 'total_scheduled_service_minutes',
       'total_vp', 'vp_in_shape', 'is_early', 'is_ontime', 'is_late',
       'n_vp_trips', 'vp_per_minute', 'pct_in_shape',
       'pct_rt_journey_atleast1_vp', 'pct_rt_journey_atleast2_vp',
       'pct_sched_journey_atleast1_vp', 'pct_sched_journey_atleast2_vp',
       'rt_sched_journey_ratio', 'avg_rt_service_minutes', 'sched_rt_category',
       'speed_mph', 'schedule_source_record_id', 'base64_url',
       'caltrans_district', 'portfolio_organization_name']

In [30]:
df2.service_date.unique()

array(['2025-01-15T00:00:00.000000000', '2025-02-12T00:00:00.000000000',
       '2025-03-12T00:00:00.000000000', '2025-04-16T00:00:00.000000000',
       '2025-05-14T00:00:00.000000000'], dtype='datetime64[ns]')

In [31]:
route_80_89 = df2.loc[(df2.portfolio_organization_name == portfolio_name)
                     & (df2.recent_combined_name == "80-89 Coastal Express")
                     & (df2.service_date == '2025-05-14T00:00:00.000000000')]

### There are two directions, 3 time periods so I should only see 6 rows per date?

In [33]:
len(route_80_89), len(route_80_89.drop_duplicates())

(239, 41)

In [35]:
route_80_89_dedup = route_80_89.drop_duplicates()

In [36]:
route_80_89_dedup.time_period.value_counts()

all_day    19
offpeak    13
peak        9
Name: time_period, dtype: int64

In [37]:
route_80_89_dedup.loc[route_80_89_dedup.time_period == "peak"]

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,service_date,is_express,is_ferry,is_rail,is_coverage,is_local,is_downtown_local,is_rapid,typology,name,combined_name,recent_combined_name,recent_route_id,route_primary_direction,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,schedule_source_record_id,base64_url,caltrans_district,portfolio_organization_name
13089,1770249a5a2e770ca90628434d4934b1,4134,0,peak,93.00,29.04,4,0.50,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,80__80-89 Coastal Express,80-89 Coastal Express,4134,Westbound,99,97,96.68,181.00,292,0,2,0,0,2,3.02,0.00,1.00,1.00,0.55,0.54,0.53,48.34,schedule_and_vp,33.15,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
13835,1770249a5a2e770ca90628434d4934b1,4134,1,peak,86.80,7.71,5,0.62,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,80__80-89 Coastal Express,80-89 Coastal Express,4134,Eastbound,466,455,589.67,346.00,1367,0,0,0,4,4,2.32,0.00,0.79,0.77,1.00,1.00,1.70,147.42,schedule_and_vp,15.47,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
13934,1770249a5a2e770ca90628434d4934b1,4136,1,peak,50.00,32.10,1,0.12,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,80X__80-89 Coastal Express,80-89 Coastal Express,4136,Eastbound,90,85,159.92,50.00,279,0,0,0,1,1,1.74,0.00,0.56,0.53,1.00,1.00,3.20,159.92,schedule_and_vp,NaN,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
14004,1770249a5a2e770ca90628434d4934b1,4137,1,peak,125.50,28.88,2,0.25,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,81__80-89 Coastal Express,80-89 Coastal Express,4137,Eastbound,78,78,77.67,134.00,234,0,1,0,0,1,3.01,0.00,1.00,1.00,0.58,0.58,0.58,77.67,schedule_and_vp,19.46,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
14109,1770249a5a2e770ca90628434d4934b1,4141,1,peak,128.00,17.82,2,0.25,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,84__80-89 Coastal Express,80-89 Coastal Express,4141,Eastbound,309,264,375.40,262.00,816,0,0,0,2,2,2.17,0.00,0.82,0.70,1.00,1.00,1.43,187.70,schedule_and_vp,NaN,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
14173,1770249a5a2e770ca90628434d4934b1,4143,1,peak,129.00,3.70,1,0.12,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,85__80-89 Coastal Express,80-89 Coastal Express,4143,Eastbound,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,schedule_only,NaN,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los Angeles / Ventura,"Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"
14237,1770249a5a2e770ca90628434d4934b1,4145,1,peak,106.00,2.91,2,0.25,2025-05-14,1.00,0.00,0.00,0.00,0.00,0.00,0.00,express,VCTC GMV Schedule,86__80-89 Coastal Express,80-89 Coastal Express,4145,Eastbound,349,341,1428.93,212.00,1029,0,0,0,2,2,0.72,0.00,0.24,0.24,1.00,1.00,6.74,714.46,schedule_and_vp,30.45,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,07 - Los A